<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tensorflow-js/tensorflow_js_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook requires a GPU resource.

In [0]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-934e6bcc-5825-c3c4-259c-6ee472a38998)


In [0]:
#@title ### Install nodejs v13.x
!sudo apt-get update -y -qq 
!sudo apt-get upgrade -y -qq 
!curl -sL https://deb.nodesource.com/setup_13.x | sudo bash -
!sudo apt-get install -y -qq nodejs
!node --version

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 55.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../base-files_10.1ubuntu2.8_amd64.deb ...
Unpacking base-files (10.1ubuntu2.8) over (10.1ubuntu2.7) ...
Setting up base-files (10.1ubuntu2.8) ...
Installing new version of config file /etc/issue ...
Installing new version of config file /etc/issue.net ...
Installing new version of config file /etc/lsb-release ...
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../bsdutils_1%3a2.31.1-0.4ubuntu3.6_amd64.deb .

In [0]:
#@title ## Setup javascript kernel
#@markdown We use ``ijavascript`` to enable a javascript kernel inside Colab. 

#@markdown ---
#@markdown **Important:** Wait until this cell's run completes and reload your 
#@markdown environment pressing ``F5``.

#@markdown ---

!npm install -g -q --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

/usr/bin/ijs -> /usr/lib/node_modules/ijavascript/bin/ijavascript.js
/usr/bin/ijsconsole -> /usr/lib/node_modules/ijavascript/bin/ijsconsole.js
/usr/bin/ijsinstall -> /usr/lib/node_modules/ijavascript/bin/ijsinstall.js
/usr/bin/ijskernel -> /usr/lib/node_modules/ijavascript/lib/kernel.js
/usr/bin/ijsnotebook -> /usr/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.2.0 install /usr/lib/node_modules/ijavascript/node_modules/zeromq
> node scripts/prebuild-install.js || (node scripts/preinstall.js && node-gyp rebuild)


prebuild-install WARN install No prebuilt binaries found (target=13.12.0 runtime=node arch=x64 libc= platform=linux)

Building libzmq for linux
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whe

# Training

In [5]:
#@title ## Install TensorFlow.js
#@markdown If you run into any "SyntaxError: invalid syntax" issue, you have
#@markdown probably not reloaded this notebook. Press ``F5`` and try to 
#@markdown execute this cell again.
 
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
sh('npm init -y');
sh('npm install @tensorflow/tfjs-node-gpu --save -q')
sh('npm install neat-csv --save -q')

Wrote to /content/package.json:

{
  "name": "content",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}





In [7]:
sh('npm install -q');


> @tensorflow/tfjs-node-gpu@1.7.2 install /content/node_modules/@tensorflow/tfjs-node-gpu
> node scripts/install.js gpu download


GPU-linux-1.7.2.tar.gz



* Downloading libtensorflow



* Building TensorFlow Node.js bindings

npm
 WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.





+ @tensorflow/tfjs-node-gpu@1.7.2
added 93 packages from 99 contributors and audited 166 packages in 18.634s



npm
 WARN content@1.0.0 No description

npm WARN
 content@1.0.0 No repository field.





audited 50 packages in 13.459s


1 package is looking for funding
  run `npm fund` for details


found 0 vulnerabilities



1 package is looking for funding
  run `npm fund` for details


found 0 vulnerabilities




## "Imports"

In [11]:
var tf = require('@tensorflow/tfjs-node-gpu')
var tfd = require('@tensorflow/tfjs-data')
var neatCsv = require('neat-csv')
var fs = require('fs')
var assert = require('assert').strict

console.log('GPU enabled:', tf.backend().isUsingGpuDevice)

GPU enabled: true


### Load dataset

In [0]:
/**
 * Load a local csv file and prepare the data for training
 * 
 * @param {string} csvPath The path to csv file.
 * @returns {tf.data.Dataset} The loaded and prepared Dataset.
 */
async function loadDataset(csvPath, hasHeader = true) { 
  const tfDatasetConfig = {
    hasHeader,
    columnConfigs: {
      'rings': {
        isLabel: true
      }
    }
  }
  const dataset = tfd.csv(csvPath, tfDatasetConfig)
  const numOfColumns = (await dataset.columnNames()).length - 1

  // const row = await dataset.take(1).toArray();
  // console.log('row ... dataset.take(1).toArray()', row)

  return {
    dataset: dataset.map(row => {
      const rawFeatures = row['xs']
      const rawLabel = row['ys']
      const convertedFeatures = Object.keys(rawFeatures).map(key => {
        switch (rawFeatures[key]) {
          case 'F':
            return 0
          case 'M':
            return 1
          case 'I':
            return 2
          default:
            return Number(rawFeatures[key])
        }
      })
      const convertedLabel = [rawLabel['rings']]
      return {xs: convertedFeatures, ys: convertedLabel}
    }),
    numOfColumns,
  }
}

Make dataset available on an object name ``data``.

In [0]:
var data

run_async(async function () {
  const CSV_PATH = 'https://storage.googleapis.com/tfjs-examples/abalone-node/abalone.csv'
  data = await loadDataset(CSV_PATH)
})

Quick smoke test to validate our dataset object.

In [39]:
assert.equal(data.numOfColumns, 8)
run_async(async function () {
  console.log(await data.dataset.take(1).toArray())
})

[
  {
    xs: [
          1,  0.455,
      0.365,  0.095,
      0.514, 0.2245,
      0.101,   0.15
    ],
    ys: [ 15 ]
  }
]


## Model

In [0]:
/**
 * Builds a MLP regression model.
 * 
 * @param {number} inputShape The input of the model.
 * @returns {tf.Sequential} The generated model
 */
function createModel(inputShape, activation = 'sigmoid', units = 50,
                     loss = 'meanSquaredError') { 
  const model = tf.sequential();
  model.add(tf.layers.dense({
    inputShape,
    activation,
    units,
  }))
  model.add(tf.layers.dense({
    activation,
    units,
  }))
  model.add(tf.layers.dense({
    units: 1,
  }))
  model.compile({
    optimizer: tf.train.sgd(0.01),
    loss,
  })
  return model
}

In [84]:
model = createModel(data.numOfColumns)

// perform a quick smoke test
assert.equal(model.inputs.length, 1)
assert.deepEqual(model.inputs[0].shape, [null, data.numOfColumns])
assert.equal(model.outputs.length, 1)
assert.deepEqual(model.outputs[0].shape, [null, 1])

model.summary()

_________________________________________________________________
Layer (type)                 Output shape              Param #   
dense_Dense34 (Dense)        [null,50]                 450       
_________________________________________________________________
dense_Dense35 (Dense)        [null,50]                 2550      
_________________________________________________________________
dense_Dense36 (Dense)        [null,1]                  51        
Total params: 3051
Trainable params: 3051
Non-trainable params: 0
_________________________________________________________________


## Train

In [200]:
run_async(async function() {
  const batchSize = 500
  // The dataset has 4177 rows. Our goal is to split them 
  // into 2 groups, one for training(3500 rows) and the other for 
  // validation (677 rows).
  const trainBatches = Math.floor(3500 / batchSize)
  const dataset = data.dataset.shuffle(1000).batch(batchSize)
  const trainDataset = dataset.take(trainBatches)
  const validationDataset = dataset.skip(trainBatches)
  // console.log(await trainDataset.take(1).toArray())
  // console.log(await validationDataset.take(1).toArray())
})


[
  {
    xs: Tensor {
      kept: false,
      isDisposedInternal: false,
      shape: [Array],
      dtype: 'float32',
      size: 4000,
      strides: [Array],
      dataId: {},
      id: 178,
      rankType: '2'
    },
    ys: Tensor {
      kept: false,
      isDisposedInternal: false,
      shape: [Array],
      dtype: 'float32',
      size: 500,
      strides: [Array],
      dataId: {},
      id: 179,
      rankType: '2'
    }
  }
]
[
  {
    xs: Tensor {
      kept: false,
      isDisposedInternal: false,
      shape: [Array],
      dtype: 'float32',
      size: 4000,
      strides: [Array],
      dataId: {},
      id: 194,
      rankType: '2'
    },
    ys: Tensor {
      kept: false,
      isDisposedInternal: false,
      shape: [Array],
      dtype: 'float32',
      size: 500,
      strides: [Array],
      dataId: {},
      id: 195,
      rankType: '2'
    }
  }
]
